In [1]:
from decouple import config
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_columns', None)

sys.path.insert(0, config('PROYECTO_DIR'))
from clases.bd.conexion import  SQLServerConnector
conn = SQLServerConnector('DBGERESA')
Anio = 2025
f_inicio=1
f_fin=12 


E_Normal = ['Z3491', 'Z3492', 'Z3493']
E_AltoRiesgo = ['Z3591', 'Z3592', 'Z3593']
Embarazo = E_Normal + E_AltoRiesgo
DosajeHb = ['85018','85018.01','80055.01','85031']
DxAnemia = ['O990','D509']
EntregaHierro = ['99199.26']
VisitaFamiliar = ['C0011' ,'99499']

todos_codigos = Embarazo + DosajeHb + DxAnemia + EntregaHierro + VisitaFamiliar

✅ Conexión exitosa a la base de datos 'DBGERESA'


<h1> Gestante </h1>


In [2]:


Envaraso = "', '".join(todos_codigos)

query = f"""
SELECT 
	pvt.Id_Cita,	
	pvt.T_DOC,
	pvt.NUMERO,
	FORMAT(pvt.F_NACIMIENTO, 'dd-MM-yyyy') AS F_NACIMIENTO,
	pvt.SEXO,
	pvt.Tipo_Edad,
	pvt.Edad_Reg,
	FORMAT(pvt.FECHA_ATENCION, 'dd-MM-yyyy') AS FECHA_ATENCION,
	pvt.T_DIAG,
	pvt.Codigo_Item,
	pvt.Descripcion_Item,
	ISNULL(pvt.[1], '') AS LAB1,
	ISNULL(pvt.[2], '') AS LAB2,
	ISNULL(pvt.[3], '') AS LAB3,
	FORMAT(pvt.F_REGISTRO, 'dd-MM-yyyy') AS F_REGISTRO,
	FORMAT(pvt.F_MODIFICACION, 'dd-MM-yyyy') AS F_MODIFICACION,
	pvt.ESTABLECIMIENTO,
	pvt.DISTRITO,
	pvt.POVINCIA

FROM (
	SELECT distinct 
		h.Id_Cita,
		d.Abrev_Tipo_Doc AS T_DOC,
		p.Numero_Documento AS NUMERO,
		p.Fecha_Nacimiento AS F_NACIMIENTO,
		p.Genero AS SEXO,
		h.Fecha_Ultima_Regla,
		h.Tipo_Edad,
		h.Edad_Reg,
		h.Fecha_Atencion AS FECHA_ATENCION,
		h.Codigo_Item,
		Descripcion_Item,		
		h.Tipo_Diagnostico AS T_DIAG,
		h.Valor_Lab AS LAB,
		h.Id_Correlativo_Lab AS N_LAB,
		h.Fecha_Registro AS F_REGISTRO,
		h.Fecha_Modificacion AS F_MODIFICACION,
		r.est_nombre AS ESTABLECIMIENTO,
		r.DESC_DIST AS DISTRITO,
		r.DESC_PROV AS POVINCIA      
    
	FROM DBGERESA.dbo.HISMINSA h
	INNER JOIN DBGERESA.dbo.MAESTRO_PACIENTE p ON p.Id_Paciente = h.Id_Paciente
	INNER JOIN DBGERESA.dbo.MAESTRO_HIS_TIPO_DOC d ON d.Id_Tipo_Documento = p.Id_Tipo_Documento
	INNER JOIN DBGERESA.dbo.RENIPRESS r ON r.COD_ESTAB = h.renipress
	left JOIN DBGERESA.dbo.MAESTRO_HIS_CIE_CPMS C ON C.Codigo_Item=H.Codigo_Item

	WHERE 
	h.Anio in({Anio},{Anio-1})       
	and h.Codigo_Item in('{Envaraso}')
	and p.Id_Tipo_Documento = 1
	) AS src
PIVOT (
	MAX(LAB)
	FOR N_LAB IN ([1], [2], [3])
) AS pvt
"""

df = conn.ejecutar_sql(query, retornar_datos=True)
Gestante = pd.DataFrame(df)
Gestante['FECHA_ATENCION'] = pd.to_datetime(Gestante['FECHA_ATENCION'], format='%d-%m-%Y', errors='coerce')
#Gestante.head()

In [24]:

#gestante_dni = Gestante[Gestante['Codigo_Item'].isin(Embaraso)][['T_DOC','NUMERO']].drop_duplicates().reset_index(drop=True).copy()
#gestante_dni.head(5)
#gestante_dni.head(5)

def obtener_gestantes_unicas(df_input, codigos_embarazo):

    gestante_dni = df_input[df_input['Codigo_Item'].isin(codigos_embarazo)][['T_DOC', 'NUMERO']] \
                   .drop_duplicates() \
                   .reset_index(drop=True) \
                   .copy()
    
    return gestante_dni
#gestante_dni = obtener_gestantes_unicas(Gestante, Embarazo)

<h1>CAPTACION DE GESTANTES</h1>

In [2]:
def generar_captacion(df_input):
    df = df_input.copy()   

    if 'LAB1' in df.columns:
        df['LAB1_NUM'] = pd.to_numeric(df['LAB1'], errors='coerce')
        df = df[df['LAB1_NUM'] == 1]     

    df['FECHA_DT'] = pd.to_datetime(df['FECHA_ATENCION'], dayfirst=True, errors='coerce')
    
    df['FECHA_ATENCION'] = df['FECHA_ATENCION'].astype(str)

    condiciones = [
        df['Codigo_Item'].isin(['Z3491', 'Z3591']), 
        df['Codigo_Item'].isin(['Z3492', 'Z3592']), 
        df['Codigo_Item'].isin(['Z3493', 'Z3593'])
    ]
    
    nombres_grupos = [
        '1 TRIMESTRE', 
        '2 TRIMESTRE', 
        '3 TRIMESTRE'
    ]
    
    df['GRUPO_TEMP'] = np.select(condiciones, nombres_grupos, default=None)
    
    df_filtrado = df[df['GRUPO_TEMP'].notnull()]
    
    df_anios = df_filtrado.groupby(['T_DOC', 'NUMERO'])['FECHA_DT'].max().reset_index()
    df_anios['ANIO'] = df_anios['FECHA_DT'].dt.year.fillna(0).astype(int).astype(str)
    df_anios['ANIO'] = df_anios['ANIO'].replace('0', '')
    
    df_anios = df_anios[['T_DOC', 'NUMERO', 'ANIO']]

    df_captacion = df_filtrado.pivot_table(
        index=['T_DOC', 'NUMERO'],
        columns='GRUPO_TEMP',
        values='FECHA_ATENCION',
        aggfunc=lambda x: ', '.join(sorted(set(x))), 
        fill_value=''
    ).reset_index()
    
    df_captacion.columns.name = None
    
    df_final = pd.merge(df_anios,df_captacion,  on=['T_DOC', 'NUMERO'], how='left')
    
    return df_final

#Captacion = generar_captacion(Gestante)
#Captacion.head(9)

In [ ]:
#Captacion.to_excel("Reporte_Gestantes.xlsx", index=False)

<h1>CONTROLES DE GESTANTE </H1>

In [26]:
def ATC_CONTROLES(df_input):
    Embaraso = E_Normal + E_AltoRiesgo
    df = df_input[df_input['Codigo_Item'].isin(Embaraso)].copy()
    
    df['FECHA_ATENCION'] = df['FECHA_ATENCION'].astype(str).str.strip().replace({'nan': '', 'None': ''})
    df['LAB1'] = df['LAB1'].astype(str).str.strip().replace({'nan': '', 'None': ''})
    df['LAB2'] = df['LAB2'].astype(str).str.strip().replace({'nan': '', 'None': ''})

    valores_permitidos = [str(i) for i in range(1, 14)]
    df = df[df['LAB1'].isin(valores_permitidos)]

    pivot = df.pivot_table(
        index=['T_DOC', 'NUMERO'],
        columns='LAB1',
        values=['FECHA_ATENCION', 'LAB2'],
        aggfunc=lambda x: ', '.join(sorted(set(v for v in x if v != ''))),
        fill_value=''
    )

    df_final = pd.DataFrame(index=pivot.index).reset_index()

    for i in range(1, 14):
        c = str(i)
        
        if ('FECHA_ATENCION', c) in pivot.columns:
            df_final[c] = pivot[('FECHA_ATENCION', c)].values
        else:
            df_final[c] = ''

        nombre_col_edad = f"Edad_G{c}"
        if ('LAB2', c) in pivot.columns:
            df_final[nombre_col_edad] = pivot[('LAB2', c)].values
        else:
            df_final[nombre_col_edad] = ''

    return df_final
#df_CONTROLES = ATC_CONTROLES(Gestante)
#df_atc.head()
#df_filtrado = df_atc[df_atc['NUMERO'] == '42334868']
#print(df_filtrado)
#df_filtrado.head()

<H1> GESTANTES CON ANEMIA </H1>

In [ ]:
#df_atc.head()

<h1>Eliminar Excluciones</h1>

In [27]:

def exclusiones(Gestante, excluye):
    # Se excluye:
    #• Las atenciones vinculadas a la visita familiar integral -VD: C0011.
    #• Las atenciones de telemedicina: 99499  
    ids_a_excluir = Gestante.loc[Gestante['Codigo_Item'].isin(excluye), 'Id_Cita'].unique()

    df_filtrado = Gestante[~Gestante['Id_Cita'].isin(ids_a_excluir)].copy()
    
    return df_filtrado
#DF_Anemia = exclusiones(Gestante, VisitaFamiliar)
#DF_Anemia.head()

In [28]:
import pandas as pd

def procesar_anemia_gestantes(df_input, cod_embarazo, cod_dosaje, cod_dx_anemia):
     
    # 1. Identificar IDs que cumplen cada condición
    ids_embarazo = set(df_input[df_input['Codigo_Item'].isin(cod_embarazo)]['Id_Cita'])
    
    ids_dosaje = set(df_input[df_input['Codigo_Item'].isin(cod_dosaje)]['Id_Cita'])
    
    ids_anemia = set(df_input[
        (df_input['Codigo_Item'].isin(cod_dx_anemia)) & 
        (df_input['T_DIAG'].astype(str).str.strip() == 'D')
    ]['Id_Cita'])

    # 2. Intersección: Solo IDs que tienen LAS TRES cosas en la misma cita
    ids_validos = ids_embarazo & ids_dosaje & ids_anemia
    
    # Validación: Si no hay coincidencias, retornar DataFrame vacío con las columnas esperadas
    if not ids_validos:
        cols_vacias = ['T_DOC', 'NUMERO', 'FECHA_ATENCION', 'codigo(Embaraso)', 
                       'codigo(DosajeHb)', 'codigo(DxAnemia)', 'T_DIAG', 'num_fila']
        return pd.DataFrame(columns=cols_vacias)

    # 3. Filtrar Dataframe Base
    df_filtrado = df_input[df_input['Id_Cita'].isin(ids_validos)].copy()

    # 4. Crear Sub-tablas para aplanar la información
    # Datos Paciente
    sub_paciente = df_filtrado[['Id_Cita', 'T_DOC', 'NUMERO', 'FECHA_ATENCION']].drop_duplicates('Id_Cita')

    # Datos Embarazo
    sub_emb = df_filtrado[df_filtrado['Codigo_Item'].isin(cod_embarazo)][['Id_Cita', 'Codigo_Item']].drop_duplicates('Id_Cita')
    sub_emb = sub_emb.rename(columns={'Codigo_Item': 'codigo(Embaraso)'})

    # Datos Dosaje
    sub_dos = df_filtrado[df_filtrado['Codigo_Item'].isin(cod_dosaje)][['Id_Cita', 'Codigo_Item']].drop_duplicates('Id_Cita')
    sub_dos = sub_dos.rename(columns={'Codigo_Item': 'codigo(DosajeHb)'})

    # Datos Diagnóstico Anemia
    sub_ane = df_filtrado[
        (df_filtrado['Codigo_Item'].isin(cod_dx_anemia)) & 
        (df_filtrado['T_DIAG'].astype(str).str.strip() == 'D')
    ][['Id_Cita', 'Codigo_Item', 'T_DIAG']].drop_duplicates('Id_Cita')
    sub_ane = sub_ane.rename(columns={'Codigo_Item': 'codigo(DxAnemia)'})

    # 5. Unir (Merge)
    df_final = sub_paciente.merge(sub_emb, on='Id_Cita', how='inner') \
                           .merge(sub_dos, on='Id_Cita', how='inner') \
                           .merge(sub_ane, on='Id_Cita', how='inner')

    # 6. Ordenar: Por paciente y fecha más reciente primero
    df_final = df_final.sort_values(by=['NUMERO', 'FECHA_ATENCION'], ascending=[True, False])

    # 7. Crear contador de filas por paciente
    df_final['num_fila'] = df_final.groupby('NUMERO').cumcount() + 1

    # 8. Selección final de columnas
    cols_finales = [
        'T_DOC', 
        'NUMERO', 
        'FECHA_ATENCION', 
        'codigo(Embaraso)', 
        'codigo(DosajeHb)', 
        'codigo(DxAnemia)', 
        'T_DIAG', 
        'num_fila'
    ]

    return df_final[cols_finales].copy()


#df_Anenia_PrimeraEntrega = procesar_anemia_gestantes(DF_Anemia, Embarazo, DosajeHb, DxAnemia)


#df_Anenia_PrimeraEntrega.head(10)

In [ ]:
gestante_dni = obtener_gestantes_unicas(Gestante, Embarazo)
Captacion = generar_captacion(Gestante)


In [3]:

df_CONTROLES = ATC_CONTROLES(Gestante)
df_CONTROLES = df_CONTROLES.sort_values(by='ANIO', ascending=False)
df_CONTROLES = df_CONTROLES.reset_index(drop=True)
nuevas_columnas = [
    ("DATOS DE LA GESTANTE", "T_DOC"),
    ("DATOS DE LA GESTANTE", "NUMERO"),
    ("CAPTACION GESTANTE", "1 TRIMESTRE"),
    ("CAPTACION GESTANTE", "2 TRIMESTRE"),
    ("CAPTACION GESTANTE", "3 TRIMESTRE"),
    ("CAPTACION GESTANTE", "ANIO")
]
df_CONTROLES.columns = pd.MultiIndex.from_tuples(nuevas_columnas)
df_CONTROLES.to_excel("Reporte_Gestantes_Estructurado.xlsx")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\IRVIN\AppData\Local\Temp\ipykernel_19148\1343221622.py", line 1, in <module>
    df_CONTROLES = ATC_CONTROLES(Gestante)
                   ^^^^^^^^^^^^^
NameError: name 'ATC_CONTROLES' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ~~~~~~~~~~~

In [ ]:
df_Anenia_PrimeraEntrega.to_excel("Reporte_Gestantes.xlsx", index=False)



In [ ]:
#check_paciente = Gestante[
#    (Gestante['NUMERO'] == '42334868') & 
#    (Gestante['LAB1'].astype(str).str.strip() == '6')
#][['FECHA_ATENCION', 'LAB1', 'LAB2', 'ESTABLECIMIENTO']]


#check_paciente.head(5)